In [3]:
class World:
    def __new__(cls):
        if hasattr(cls, 'instance'):
            return cls.instance
        else:
            cls.instance = super().__new__(cls)
            return cls.instance


w1 = World()
w2 = World()
print(id(w1), id(w2))

1962685286784 1962685286784


#### Decorators in Python

A syntax feature that aids in AOP patterns in a simpler syntax / structure 

In [12]:
def to_upper(fn):
    def wrap():
        return fn().upper()
    return wrap

def strong(fn):
    def wrap():
        return "<strong>" + fn() + "</strong>"
    return wrap




In [ ]:
@strong
@to_upper
def greet():
    return "Hello, world"

#greet = to_upper(greet)
greet()

'<strong>HELLO, WORLD</strong>'

In [30]:
def decorator(fn):
    print("decorator invoked")
    def wrap():
        print("wrap invoked")
        fn()
    return wrap

@decorator
def testfn():
    print("testfn invoked")

print("start")
testfn()


decorator invoked
start
wrap invoked
testfn invoked


In [ ]:
def profile(fn):
    from time import time, ctime
    stats = {}
    def wrap(*args, **kwargs):
        start = time()

        ret = fn(*args, **kwargs)
        
        duration = time() - start
        record = (args, kwargs, ctime(), duration)
        stats.setdefault(fn.__qualname__, []).append(record)
        
        return ret

    def report():
        for rec in stats.get(fn.__qualname__, []):
            args, kwargs, ts, duration = rec
            print(f"{ts}: {fn.__qualname__}{args} took {duration} seconds.")    

    wrap.report = report
    return wrap

@profile
def slow_test(count):
    for i in range(count):
        pass
    return count * 2

print(slow_test(100_000_000))
print(slow_test(10_000_000))

# OUT: Wed Aug 13 10:43:15 2025: slow_test(100_000_000) took 1.23 seconds
# OUT  Wed Aug 13 10:43:16 2025: slow_test(10_000_000) took 0.27 seconds


200000000
20000000


In [54]:
class Profile:
    def __init__(self, fn):
        self.stats = {}
        self.fn = fn

    def __call__(self, *args, **kwargs):
        from time import time, ctime
        start = time()

        ret = self.fn(*args, **kwargs)
        duration = time() - start
        
        record = (args, kwargs, ctime(), duration)
        self.stats.setdefault(self.fn.__qualname__, []).append(record)
        
        return ret

    def report(self):
        for rec in self.stats.get(self.fn.__qualname__, []):
            args, kwargs, ts, duration = rec
            print(f"{ts}: {self.fn.__qualname__}{args} took {duration} seconds.")    

@Profile
def slow_test(count):
    for i in range(count):
        pass
    return count * 2

print(slow_test(100_000_000))
print(slow_test(10_000_000))

# OUT: Wed Aug 13 10:43:15 2025: slow_test(100_000_000) took 1.23 seconds
# OUT  Wed Aug 13 10:43:16 2025: slow_test(10_000_000) took 0.27 seconds


200000000
20000000


In [55]:
slow_test.report()

Wed Aug 13 12:08:43 2025: slow_test(100000000,) took 1.6132164001464844 seconds.
Wed Aug 13 12:08:43 2025: slow_test(10000000,) took 0.17212653160095215 seconds.


In [20]:
from time import ctime
ctime()

'Wed Aug 13 10:43:15 2025'

In [28]:
def style(s):
    def decorate(fn):
        if s == "strong":
            def wrap():
                return "<strong>" + fn() + "</strong>"
        elif s == "upper":
            def wrap():
                return fn().upper()
        else:
            wrap = fn
        return wrap
    return decorate

@style("upper")
def greet():
    return "Hello, world"

greet()

'HELLO, WORLD'

In [40]:
class Car:
    def __init__(self, make):
        self.make = make

    def __add__(self, other):
        return Car(self.make + " " + other.make)
    
    def drive(self):
        print(f"Driving {self.make} car")
    
c1 = Car("Maruti")
c2 = Car("Suzuki")
c3 = c1 + c2 # c1.__add__(c2) -> Car.__add__(c1, c2)
c3.drive()      

Driving Maruti Suzuki car


In [46]:
class ToUpper:
    def __init__(self, target):
        self.target = target

    def __call__(self):
        return self.target()
    
    def __str__(self):
        #return str(self.target)  # Bad practice
        return f"<@Toupper: {str(self.target)}>"
@ToUpper
def greet():
    return "Hello, world"

# greet = ToUpper(greet)

greet()
print(greet, type(greet))
greet()

<@Toupper: <function greet at 0x000001C8F9B422A0>> <class '__main__.ToUpper'>


'Hello, world'

In [34]:
def greet(): 
    return "Hello, world"


type(greet), id(greet)

(function, 1962693403296)

In [ ]:
class Style:
    def __init__(self, s):
        self.style = s

    def __call__(self, fn):
        self.target = fn
        if self.style == 'upper':
            return self.to_upper
        elif self.style == 'strong':
            return self.to_strong
        
    def to_upper(self):
        return self.target().upper()
    
    def to_strong(self):
        return "<strong>" + self.target() + "</strong>"
    
@Style("upper")
@Style("strong")
def greet():
    return "Hello, world"

greet()

'<STRONG>HELLO, WORLD</STRONG>'

In [52]:
class Style:
    def __init__(self, s):
        self.style = s

    def __call__(self, fn):
        self.target = fn
        wrapper = f'to_{self.style}'
        if hasattr(self, wrapper):
            return getattr(self, wrapper)
        else:
            return self.target
           
    def to_upper(self):
        return self.target().upper()
    
    def to_strong(self):
        return "<strong>" + self.target() + "</strong>"
    
    def to_italics(self):
        return "<i>" + self.target() + "</i>"
    
@Style("upper")
@Style("strong")
@Style("italics")
def greet():
    return "Hello, world"

greet()

'<STRONG><I>HELLO, WORLD</I></STRONG>'

In [2]:
# Memoize Pattern (Memento)

class Memoize:
    def __init__(self, fn):
        self.fn = fn
        self.cache = {}

    def __call__(self, *args, **kwargs):
        key = self.fn.__qualname__ + str(args) + str(kwargs)
        if key not in self.cache:
            self.cache[key] = self.fn(*args, **kwargs)
        return self.cache[key]
    

from time import time, sleep

@Memoize
def square(x):
    sleep(1)
    print("Square of", x, "is", x*x)
    return x*x

nums = [2, 5, 2, 6, 3, 5, 2, 5, 2, 5]
result = []

start = time()
for v in nums:
    result.append(square(v))
duration = time() - start
print(duration)
print(result)


Square of 2 is 4
Square of 5 is 25
Square of 6 is 36
Square of 3 is 9
4.002784013748169
[4, 25, 4, 36, 9, 25, 4, 25, 4, 25]


In [7]:
from time import time, sleep

urls = [
    "https://python.org",
    "https://pypi.org",
    "https://fsf.org",
    "https://python.org",
    "https://python.org",
    "https://fsf.org",
    "https://www.chandrashekar.info",
    "https://python.org"
]

from functools import lru_cache

@lru_cache
def fetch_url(u):
    from urllib.request import urlopen
    print("Fetching", u)
    res = urlopen(u)
    return res.code

start = time()
result = []
for u in urls:
    r = fetch_url(u)
    result.append(r)
duration = time() - start
print(result)
print(duration)


Fetching https://python.org
Fetching https://pypi.org
Fetching https://fsf.org
Fetching https://www.chandrashekar.info
[200, 200, 200, 200, 200, 200, 200, 200]
3.732240915298462
